In [25]:
import requests
import pandas as pd

In [26]:
# * 百度坐标系 (BD-09) 与 火星坐标系 (GCJ-02)的转换
# * 即 百度 转 谷歌、高德
# * @param bd_lon
# * @param bd_lat
# * @returns {*[]}
# */
import math
def bd09togcj02(bd_lon, bd_lat):
    x_pi = 3.14159265358979324 * 3000.0 / 180.0
    x = bd_lon - 0.0065
    y = bd_lat - 0.006
    z = math.sqrt(x * x + y * y) - 0.00002 * math.sin(y * x_pi)
    theta = math.atan2(y, x) - 0.000003 * math.cos(x * x_pi)
    gg_lng = z * math.cos(theta)
    gg_lat = z * math.sin(theta)
    return [gg_lng, gg_lat]


# * 火星坐标系 (GCJ-02) 与百度坐标系 (BD-09) 的转换
# * 即谷歌、高德 转 百度
# */
def gcj02tobd09(lng, lat):
    x_PI = 3.14159265358979324 * 3000.0 / 180.0
    z = math.sqrt(lng * lng + lat * lat) + 0.00002 * math.sin(lat * x_PI)
    theta = math.atan2(lat, lng) + 0.000003 * math.cos(lng * x_PI)
    bd_lng = z * math.cos(theta) + 0.0065
    bd_lat = z * math.sin(theta) + 0.006
    return [bd_lng, bd_lat]


# wgs84转高德
def wgs84togcj02(lng, lat):
    PI = 3.1415926535897932384626
    ee = 0.00669342162296594323
    a = 6378245.0
    dlat = transformlat(lng - 105.0, lat - 35.0)
    dlng = transformlng(lng - 105.0, lat - 35.0)
    radlat = lat / 180.0 * PI
    magic = math.sin(radlat)
    magic = 1 - ee * magic * magic
    sqrtmagic = math.sqrt(magic)
    dlat = (dlat * 180.0) / ((a * (1 - ee)) / (magic * sqrtmagic) * PI)
    dlng = (dlng * 180.0) / (a / sqrtmagic * math.cos(radlat) * PI)
    mglat = lat + dlat
    mglng = lng + dlng
    return [mglng, mglat]


# GCJ02/谷歌、高德 转换为 WGS84 gcj02towgs84
def gcj02towgs84(localStr):
    lng = float(localStr.split(',')[0])
    lat = float(localStr.split(',')[1])
    PI = 3.1415926535897932384626
    ee = 0.00669342162296594323
    a = 6378245.0
    dlat = transformlat(lng - 105.0, lat - 35.0)
    dlng = transformlng(lng - 105.0, lat - 35.0)
    radlat = lat / 180.0 * PI
    magic = math.sin(radlat)
    magic = 1 - ee * magic * magic
    sqrtmagic = math.sqrt(magic)
    dlat = (dlat * 180.0) / ((a * (1 - ee)) / (magic * sqrtmagic) * PI)
    dlng = (dlng * 180.0) / (a / sqrtmagic * math.cos(radlat) * PI)
    mglat = lat + dlat
    mglng = lng + dlng
    return [lng * 2 - mglng,lat * 2 - mglat]


def transformlat(lng, lat):
    PI = 3.1415926535897932384626
    ret = -100.0 + 2.0 * lng + 3.0 * lat + 0.2 * lat * \
          lat + 0.1 * lng * lat + 0.2 * math.sqrt(abs(lng))
    ret += (20.0 * math.sin(6.0 * lng * PI) + 20.0 *
            math.sin(2.0 * lng * PI)) * 2.0 / 3.0
    ret += (20.0 * math.sin(lat * PI) + 40.0 *
            math.sin(lat / 3.0 * PI)) * 2.0 / 3.0
    ret += (160.0 * math.sin(lat / 12.0 * PI) + 320 *
            math.sin(lat * PI / 30.0)) * 2.0 / 3.0
    return ret


def transformlng(lng, lat):
    PI = 3.1415926535897932384626
    ret = 300.0 + lng + 2.0 * lat + 0.1 * lng * lng + \
          0.1 * lng * lat + 0.1 * math.sqrt(abs(lng))
    ret += (20.0 * math.sin(6.0 * lng * PI) + 20.0 *
            math.sin(2.0 * lng * PI)) * 2.0 / 3.0
    ret += (20.0 * math.sin(lng * PI) + 40.0 *
            math.sin(lng / 3.0 * PI)) * 2.0 / 3.0
    ret += (150.0 * math.sin(lng / 12.0 * PI) + 300.0 *
            math.sin(lng / 30.0 * PI)) * 2.0 / 3.0
    return ret


def bd09towgs84(bd_lon, bd_lat):
    gcj02 = bd09togcj02(bd_lon, bd_lat)
    wgs84 = gcj02towgs84(str(gcj02[0]) + "," + str(gcj02[1]))
    return wgs84


In [ ]:
# 石家庄经开区西区四至坐标
# 西至114.665974,38.024141
# 北至114.705643,38.097668
# 东至114.772477,38.015783
# 南至114.733706,37.987207

# 石家庄经开区东区四至
# 西至114.870284,38.041848
# 北至114.95027,38.047645
# 东至114.957366,38.033407
# 南至114.900144,38.023309

### 根据四至进行矩形坐标范围的对半裁切，从而进行poi获取，目标是获取的poi数量在100以内

In [41]:
# 输入的dict是north，south，west，east的字典，每个key的值为对应的经纬度
# 输出的是将一个矩形平均划分成四份，返回四个四至
def poi_area_cut(dict):
    
    north_south=round((dict['north']-dict['south'])/2,6)
    east_west=round((dict['east']-dict['west'])/2,6)
    mid_n_s=round(dict['north']-north_south,6)
    mid_e_w=round(dict['east']-east_west,6)
    
    return {'south':mid_n_s,'west':dict['west'],'north':dict['north'],
            'east':mid_e_w},{'south':mid_n_s,'west':mid_e_w,'north':dict['north'],
                             'east':dict['east']},{'south':dict['south'],'west':dict['west'],'north':mid_n_s,
                                                   'east':mid_e_w},{'south':dict['south'],'west':mid_e_w,'north':mid_n_s,'east':dict['east']}

# 从百度api获取json
def get_json(four_loc,ak,scope_status,page_num):
    params = {
        "query":"公司企业",
        "bounds":four_loc,
        "output":"json",
        "ak":ak,
        "tag":"公司企业",
        "extensions_adcode":True,
        "scope":scope_status,
        "ret_coordtype":"gcj02ll",
        "page_size":20,
        "page_num":page_num
    }
    
    url = "https://api.map.baidu.com/place/v2/search"
    response1 = requests.get(url=url, params=params)
    if response1:
        return response1.json()
    else:
        return 'error'
    
# 对坐标进行切分并获取返回结果
def get_cut_json(dict_loc):
    dict_i_1,dict_i_2,dict_i_3,dict_i_4=poi_area_cut(dict_loc)
    ls_dict_area=[dict_i_1,dict_i_2,dict_i_3,dict_i_4]
    ls_result=[]
    for j in [dict_i_1,dict_i_2,dict_i_3,dict_i_4]:
        four_loc=str(j['south'])+','+str(j['west'])+','+str(j['north'])+','+str(j['east'])
        page_num=0
        json_re=get_json(four_loc,ak,scope_status,page_num)
        ls_result.append(json_re)
    return ls_dict_area,ls_result

In [38]:
url = "https://api.map.baidu.com/place/v2/search"
ak = "hFpVwfn5Q1ZCwMkZ4yd9FyJa96OlH6Li"
scope_status="2"
page_num=0
dict_={'south': 37.987207, 'west': 114.665974, 'north': 38.042437, 'east': 114.719226}
four_loc=str(dict_['south'])+','+str(dict_['west'])+','+str(dict_['north'])+','+str(dict_['east'])
get_json(four_loc,ak,scope_status,page_num)

{'status': 0,
 'message': 'ok',
 'total': 106,
 'result_type': 'poi_type',
 'results': [{'name': '河北太行机械工业有限公司',
   'location': {'lat': 38.020003519606, 'lng': 114.70475050467},
   'address': '河北省石家庄市藁城区赣江路2号',
   'province': '河北省',
   'city': '石家庄市',
   'area': '藁城区',
   'street_id': '7af583684365f2c7a8f729ef',
   'telephone': '(0311)83092638',
   'detail': 1,
   'uid': '7af583684365f2c7a8f729ef',
   'detail_info': {'tag': '公司企业;公司',
    'navi_location': {'lng': 114.70662029373, 'lat': 38.017722029417},
    'type': 'enterprise',
    'detail_url': 'http://api.map.baidu.com/place/detail?uid=7af583684365f2c7a8f729ef&output=html&source=placeapi_v2',
    'overall_rating': '5',
    'comment_num': '2',
    'shop_hours': '',
    'label': '公司',
    'children': []}},
  {'name': '科威药业有限公司',
   'location': {'lat': 38.013886990305, 'lng': 114.68010333693},
   'address': '石家庄市藁城区丰产路博云大厦西侧约210米',
   'province': '河北省',
   'city': '石家庄市',
   'area': '藁城区',
   'street_id': '17a0396f67747d5096f08c2c',
 

In [42]:
# ak 
# 总体坐标顺序['south'],['west'],['north'],['east']
url = "https://api.map.baidu.com/place/v2/search"
ak = "hFpVwfn5Q1ZCwMkZ4yd9FyJa96OlH6Li"
scope_status="2"

dict_west={'north':38.097668,'south':37.987207,'east':114.772477,'west':114.665974}
dict_east={'north':38.047645,'south':38.023309,'east':114.957366,'west':114.870284}

ls_loc=[dict_west,dict_east]
ls_name=['west','east']
for i in range(len(ls_loc)):
    print(ls_name[i])
    df_total_result_i=pd.DataFrame()
    ls_area_i,ls_i=get_cut_json(ls_loc[i])
    for a in range(len(ls_i)):
        four_loc=str(ls_area_i[a]['south'])+','+str(ls_area_i[a]['west'])+','+str(ls_area_i[a]['north'])+','+str(ls_area_i[a]['east'])
        if ls_i[a]['total']<=100:
            
            if ls_i[a]['result_type']=='poi_type':
                print('add',ls_i[a]['total'])
                page_total=round(ls_i[a]['total']/20)+1
                for b in range(page_total):
                    page_num_b=b
                    dict_re_b=get_json(str(ls_area_i[a]['south'])+','+str(ls_area_i[a]['west'])+','+str(ls_area_i[a]['north'])+','+str(ls_area_i[a]['east']),ak,scope_status,page_num_b)
                    df_b=pd.DataFrame.from_dict(dict_re_b['results'])
                    df_total_result_i=pd.concat([df_total_result_i,df_b],axis=0)
            else:
                print('city_region',ls_area_i[a])
        else:
            print('aaa',ls_area_i[a])
            print(ls_i[a]['total'])
            ls_area_a,ls_a=get_cut_json(ls_area_i[a])
            for c in range(len(ls_a)):
                if ls_a[c]['total']<=100:
                    
                    if ls_a[c]['result_type']=='poi_type':
                        print('add_aaa',ls_a[c]['total'])
                        page_total=round(ls_a[c]['total']/20)+1
                        for b in range(page_total):
                            page_num_b=b
                            dict_re_b=get_json(str(ls_area_a[c]['south'])+','+str(ls_area_a[c]['west'])+','+str(ls_area_a[c]['north'])+','+str(ls_area_a[c]['east']),ak,scope_status,page_num_b)
                            df_b=pd.DataFrame.from_dict(dict_re_b['results'])
                            df_total_result_i=pd.concat([df_total_result_i,df_b],axis=0)
                    else:
                        print('city_region_aaa',ls_area_a[c])
                else:
                    print('bbb',ls_area_a[c])
                    print(ls_a[c]['total'])
                    ls_area_d,ls_d=get_cut_json(ls_area_a[c])
                    for d in range(len(ls_d)):
                        if ls_d[d]['total']<=100:
                            
                            if ls_d[d]['result_type']=='poi_type':
                                print('add_bbb',ls_d[d]['total'],ls_area_d[d])
                                page_total=round(ls_d[d]['total']/20)+1
                                for b in range(page_total):
                                    page_num_b=b
                                    dict_re_b=get_json(str(ls_area_d[d]['south'])+','+str(ls_area_d[d]['west'])+','+str(ls_area_d[d]['north'])+','+str(ls_area_d[d]['east']),ak,scope_status,page_num_b)
                                    df_b=pd.DataFrame.from_dict(dict_re_b['results'])
                                    df_total_result_i=pd.concat([df_total_result_i,df_b],axis=0)
                            else:
                                print('city_region_bbb',ls_area_d[d])
                        else:
                            print('ccc',ls_area_d[d])
                            print(ls_d[d]['total'])
    
    df_total_result_i.to_csv('E:\\project_data\\23shijiazhuang\\poi\\'+ls_name[i]+'_1.csv',index=False)

west
add 86
add 77
aaa {'south': 37.987207, 'west': 114.665974, 'north': 38.042437, 'east': 114.719226}
107
bbb {'south': 38.014822, 'west': 114.665974, 'north': 38.042437, 'east': 114.6926}
103
add_bbb 37 {'south': 38.02863, 'west': 114.665974, 'north': 38.042437, 'east': 114.679287}
add_bbb 43 {'south': 38.02863, 'west': 114.679287, 'north': 38.042437, 'east': 114.6926}
add_bbb 51 {'south': 38.014822, 'west': 114.665974, 'north': 38.02863, 'east': 114.679287}
add_bbb 43 {'south': 38.014822, 'west': 114.679287, 'north': 38.02863, 'east': 114.6926}
bbb {'south': 38.014822, 'west': 114.6926, 'north': 38.042437, 'east': 114.719226}
111
add_bbb 76 {'south': 38.02863, 'west': 114.6926, 'north': 38.042437, 'east': 114.705913}
add_bbb 33 {'south': 38.02863, 'west': 114.705913, 'north': 38.042437, 'east': 114.719226}
add_bbb 88 {'south': 38.014822, 'west': 114.6926, 'north': 38.02863, 'east': 114.705913}
add_bbb 33 {'south': 38.014822, 'west': 114.705913, 'north': 38.02863, 'east': 114.719226

In [73]:
d={'south': 38.023309, 'west': 114.913825, 'north': 114.913825, 'east': 114.957366}
str(d['south'])+','+str(d['west'])+','+str(d['north'])+','+str(d['east'])

'38.023309,114.913825,114.913825,114.957366'

### 根据地址反查坐标

In [67]:
# 根据地址反查坐标
def get_lng_lat(str_adr,city):
    url = "https://api.map.baidu.com/geocoding/v3"
    ak = "hFpVwfn5Q1ZCwMkZ4yd9FyJa96OlH6Li"

    params = {
        "address":str_adr,
        "output":"json",
        "ak":ak,
        "city":city,
        "ret_coordtype":"gcj02",
        "extension_analys_level":1
    }

    response = requests.get(url=url, params=params)
    if response:
        return response.json()
    else:
        return 'error'

In [71]:
city='石家庄市'
stradr='藁城经济技术开发区，规划振兴街东侧，石德铁路南侧。'
result=get_lng_lat(stradr,city)

In [73]:
result

{'status': 0,
 'result': {'location': {'lng': 114.80476708988719, 'lat': 38.14784911109317},
  'precise': 0,
  'confidence': 30,
  'comprehension': 100,
  'level': '道路',
  'analys_level': 'POI'}}

In [74]:
result['result']['location']

{'lng': 114.80476708988719, 'lat': 38.14784911109317}

In [75]:
result['status']

0

### 国测局gcj02转84坐标

In [18]:
locstr='114.66437666317,37.985946385272'
gcj02towgs84(locstr)

[114.65852068252006, 37.985376064086985]

In [76]:
# baidu poi
df_bdpoi=pd.read_excel(r'E:\project_data\23shijiazhuang\poi\total.xlsx',sheet_name='dixiao')
# ls=df_bdpoi['location'].values.tolist()
# df_bdpoi.loc[:,['lat','lng']]=pd.DataFrame.from_dict([eval(x) for x in ls])

df_bdpoi.loc[:,'lng_lat']=df_bdpoi.apply(lambda row: f'{row["lng"]},{row["lat"]}', axis=1)
for i in range(len(df_bdpoi)):
    if type(df_bdpoi.loc[i,'lng_lat'])==str:
        df_bdpoi.loc[i,['lng_84','lat_84']]=gcj02towgs84(df_bdpoi.loc[i,'lng_lat'])

In [78]:
df_bdpoi.to_csv(r'E:\project_data\23shijiazhuang\poi\dixia_84_new.csv',index=False)

### 规上企业数据合并

In [97]:
df_total=pd.read_excel(r'E:\project_data\23shijiazhuang\poi\total.xlsx',sheet_name='total')
df_guishang=pd.read_excel(r'E:\project_data\23shijiazhuang\poi\total.xlsx',sheet_name='Sheet1')
ls=df_total['location'].values.tolist()
df_total.loc[:,['lat','lng']]=pd.DataFrame.from_dict([eval(x) for x in ls])
df_total['name'].apply(lambda x:x.split('-'))
df_total.loc[:,'format_name']=df_total['name'].apply(lambda x:x.split('-')[0])

In [98]:
df_total.to_csv('E:\\project_data\\23shijiazhuang\\poi\\total.csv',index=False)

In [99]:
df_guishang

,类别,组织机构代码,企业名称,类别.1,所在地,1-12月;工业总产值(当年价格);万元,上年同期;1-本月;工业总产值(当年价格);万元,Unnamed: 7
0,规模以上工业企业,784057215,河北白沙烟草有限责任公司,规模以上工业企业,经开区,1116563.794,973959.954,NaN
1,规模以上工业企业,601289268,石药集团欧意药业有限公司,规模以上工业企业,经开区,1439229.500,1467112.100,NaN
2,规模以上工业企业,746895357,石药集团恩必普药业有限公司,规模以上工业企业,经开区,841662.300,673250.500,"{'lat': 38.028887078472, 'lng': 114.70708783409}"
3,规模以上工业企业,689281702,河北中石油昆仑天然气有限公司,规模以上工业企业,经开区,343237.931,279270.348,NaN
4,规模以上工业企业,601908022,石药集团中诺药业（石家庄）有限公司,规模以上工业企业,经开区,353849.300,255483.500,NaN
...,...,...,...,...,...,...,...,...
122,规模以上工业企业,347660673,河北钰隆堂药业有限公司,规模以上工业企业,经开区,0.000,1513.203,NaN
123,规模以上工业企业,107912069,河北中农博远农业装备有限公司,规模以上工业企业,经开区,0.000,0.000,NaN
124,规模以上工业企业,320116088,河北天竹纺织科技有限公司,规模以上工业企业,东区,0.000,26513.134,"{'lat': 38.036442170791, 'lng': 114.87794431294}"
125,规模以上工业企业,058163320,石家庄市藁城区承浩公路工程建筑有限公司,规模以上工业企业,岗上镇,43.001,610.845,NaN


In [92]:
df_total['name'].apply(lambda x:x.split('-'))

0              [河北铁科翼辰新材科技有限公司]
1                  [天人集团农机装备公司]
2                  [河北德瑞化工有限公司]
3               [石家庄轴设机电设备有限公司]
4                [河北永泰化工集团有限公司]
                 ...           
626    [科鑫三仔广告照相图文办公24小时(北乐乡店)]
627             [石家庄海天陶瓷制品有限公司]
628                    [大运重卡新厂]
629                   [鹏帅书画装裱厂]
630                      [环东商贸]
Name: name, Length: 631, dtype: object

In [90]:
# 石家庄市 1301
# 2024年全量企业数据读取。
df_chrunks=pd.read_csv(r'E:\project_basedata\2023090601启信宝企业统计数据\t_enterprise_all_2024.csv',dtype='str',chunksize=200000)
# 企业大文件处理
df_result=pd.DataFrame()
# count=1
for chrunk in df_chrunks:
    
    chrunk.loc[:,'location']=chrunk['credit_no'].str[2:6]
    
    df_i=chrunk.loc[chrunk['location']=='1301']
    # chrunk.groupby(['location','industry_code'])['credit_no'].count().reset_index()
    df_result=pd.concat([df_result,df_i],axis=0)

In [39]:
df_result

NameError: name 'df_result' is not defined

In [85]:
ls=df_total['location'].values.tolist()
[eval(x) for x in ls]

[{'lat': 38.031811693029, 'lng': 114.87023254909},
 {'lat': 38.034247797814, 'lng': 114.86906526738},
 {'lat': 38.037217191809, 'lng': 114.89473111558},
 {'lat': 38.036008037793, 'lng': 114.86716986169},
 {'lat': 38.034776957976, 'lng': 114.89395880993},
 {'lat': 38.034811787809, 'lng': 114.88831067375},
 {'lat': 38.040893921474, 'lng': 114.88676025675},
 {'lat': 38.036190581924, 'lng': 114.89044266718},
 {'lat': 38.036442170791, 'lng': 114.87794431294},
 {'lat': 38.036775890449, 'lng': 114.89229631302},
 {'lat': 38.040105502605, 'lng': 114.88585956576},
 {'lat': 38.037475603964, 'lng': 114.86938420775},
 {'lat': 38.039054553015, 'lng': 114.89395062759},
 {'lat': 38.035988910874, 'lng': 114.86547782615},
 {'lat': 38.032603230493, 'lng': 114.87704396426},
 {'lat': 38.034304221374, 'lng': 114.89329354136},
 {'lat': 38.038815479706, 'lng': 114.89324223472},
 {'lat': 38.03944731116, 'lng': 114.88921014895},
 {'lat': 38.036561339584, 'lng': 114.88591885169},
 {'lat': 38.036098398115, 'lng':

In [86]:
pd.DataFrame.from_dict([eval(x) for x in ls])

,lat,lng
0,38.031812,114.870233
1,38.034248,114.869065
2,38.037217,114.894731
3,38.036008,114.867170
4,38.034777,114.893959
...,...,...
626,38.005158,114.665847
627,37.996315,114.665445
628,37.999523,114.680751
629,37.999862,114.666975


In [77]:
df_total.loc[:,['lat','lng']]=df_total['location'].apply(pd.Series)
df_total

,name,所在地,1-12月;工业总产值(当年价格);万元,location,address,province,city,area,street_id,detail,uid,detail_info,telephone,lat,lng
0,河北铁科翼辰新材科技有限公司,东区,36347.700,"{'lat': 38.031811693029, 'lng': 114.87023254909}",河北省石家庄市藁城区翼辰北街1号,河北省,石家庄市,藁城区,97de5c5ff0076e1d14c8f63c,1,97de5c5ff0076e1d14c8f63c,"{'tag': '公司企业;公司', 'navi_location': {'lng': 11...",NaN,NaN,NaN
1,天人集团农机装备公司,NaN,NaN,"{'lat': 38.034247797814, 'lng': 114.86906526738}",河北省石家庄市藁城区市府东路288号,河北省,石家庄市,藁城区,37d60a4605ff7c631e1d11ed,1,37d60a4605ff7c631e1d11ed,"{'tag': '公司企业;公司', 'navi_location': {'lng': 11...",NaN,NaN,NaN
2,河北德瑞化工有限公司,东区,43800.151,"{'lat': 38.037217191809, 'lng': 114.89473111558}",河北省石家庄市藁城区1号路1号,河北省,石家庄市,藁城区,NaN,1,8fa14fc350eb786df7eafced,"{'tag': '公司企业;公司', 'navi_location': {'lng': 11...","(0311)86597761,(0311)86599766",NaN,NaN
3,石家庄轴设机电设备有限公司,NaN,NaN,"{'lat': 38.036008037793, 'lng': 114.86716986169}",河北省石家庄市藁城区市府东路212号,河北省,石家庄市,藁城区,2252019d6f8828e1cae33c3c,1,2252019d6f8828e1cae33c3c,"{'tag': '公司企业;公司', 'navi_location': {'lng': 11...",13803395732,NaN,NaN
4,河北永泰化工集团有限公司,NaN,NaN,"{'lat': 38.034776957976, 'lng': 114.89395880993}",河北省石家庄市藁城区振兴北街11号,河北省,石家庄市,藁城区,8bebb7d602ff9533714b8753,1,8bebb7d602ff9533714b8753,"{'tag': '公司企业;公司', 'navi_location': {'lng': 11...",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
626,科鑫三仔广告照相图文办公24小时(北乐乡店),NaN,NaN,"{'lat': 38.005157572671, 'lng': 114.66584674116}",河北省石家庄市藁城区新丰街第三巷5号,河北省,石家庄市,藁城区,NaN,17897426608,1,de0eb38781e733427ec87603,"{'tag': '公司企业;公司', 'type': 'enterprise', 'deta...",NaN,NaN
627,石家庄海天陶瓷制品有限公司,NaN,NaN,"{'lat': 37.996315296155, 'lng': 114.66544486951}",石家庄市藁城区富康西路与新丰街交叉路口往南约240米,河北省,石家庄市,藁城区,NaN,NaN,1,4d77e9a00f23f09693114c43,"{'tag': '购物;家居建材', 'navi_location': {'lng': 11...",NaN,NaN
628,大运重卡新厂,NaN,NaN,"{'lat': 37.999522924109, 'lng': 114.68075123466}",河北省石家庄市藁城区东三环辅路,河北省,石家庄市,藁城区,NaN,NaN,1,678a0cef0370d1f3e115f989,"{'tag': '公司企业;厂矿', 'navi_location': {'lng': 11...",NaN,NaN
629,鹏帅书画装裱厂,NaN,NaN,"{'lat': 37.999862101715, 'lng': 114.66697491811}",石家庄市藁城区新丰街南乐乡小学西南侧约260米,河北省,石家庄市,藁城区,NaN,NaN,1,400b41024fe1217d5e17c3ed,"{'tag': '公司企业;厂矿', 'navi_location': {'lng': 11...",NaN,NaN


In [75]:
df_guishang

,类别,组织机构代码,企业名称,类别.1,所在地,1-12月;工业总产值(当年价格);万元,上年同期;1-本月;工业总产值(当年价格);万元,Unnamed: 7
0,规模以上工业企业,784057215,河北白沙烟草有限责任公司,规模以上工业企业,经开区,1116563.794,973959.954,NaN
1,规模以上工业企业,601289268,石药集团欧意药业有限公司,规模以上工业企业,经开区,1439229.500,1467112.100,NaN
2,规模以上工业企业,746895357,石药集团恩必普药业有限公司,规模以上工业企业,经开区,841662.300,673250.500,"{'lat': 38.028887078472, 'lng': 114.70708783409}"
3,规模以上工业企业,689281702,河北中石油昆仑天然气有限公司,规模以上工业企业,经开区,343237.931,279270.348,NaN
4,规模以上工业企业,601908022,石药集团中诺药业（石家庄）有限公司,规模以上工业企业,经开区,353849.300,255483.500,NaN
...,...,...,...,...,...,...,...,...
122,规模以上工业企业,347660673,河北钰隆堂药业有限公司,规模以上工业企业,经开区,0.000,1513.203,NaN
123,规模以上工业企业,107912069,河北中农博远农业装备有限公司,规模以上工业企业,经开区,0.000,0.000,NaN
124,规模以上工业企业,320116088,河北天竹纺织科技有限公司,规模以上工业企业,东区,0.000,26513.134,"{'lat': 38.036442170791, 'lng': 114.87794431294}"
125,规模以上工业企业,058163320,石家庄市藁城区承浩公路工程建筑有限公司,规模以上工业企业,岗上镇,43.001,610.845,NaN


In [66]:
ls_d

[{'status': 0,
  'message': 'ok',
  'total': 75,
  'result_type': 'poi_type',
  'results': [{'name': '华北制药河北华民药业有限责任公司奥奇德工厂',
    'location': {'lat': 38.024403926721, 'lng': 114.69387255047},
    'address': '河北省石家庄市藁城区扬子路18号',
    'province': '河北省',
    'city': '石家庄市',
    'area': '藁城区',
    'street_id': '',
    'telephone': '(0311)83096007',
    'detail': 1,
    'uid': '073ba6522cb0b26b97031925',
    'detail_info': {'tag': '公司企业;公司',
     'navi_location': {'lng': 114.69377524417, 'lat': 38.024293871976},
     'type': 'enterprise',
     'detail_url': 'http://api.map.baidu.com/place/detail?uid=073ba6522cb0b26b97031925&output=html&source=placeapi_v2',
     'overall_rating': '4.7',
     'comment_num': '3',
     'shop_hours': '',
     'label': '厂矿',
     'children': []}},
   {'name': '华北制药河北莱欣药业有限公司',
    'location': {'lat': 38.023034963693, 'lng': 114.68928440751},
    'address': '河北省石家庄市藁城区海南路58号',
    'province': '河北省',
    'city': '石家庄市',
    'area': '藁城区',
    'street_id': '2067f1a5eb

In [63]:
page_total

1

In [61]:
str(ls_area_d[d]['south'])+','+str(ls_area_d[d]['west'])+','+str(ls_area_d[d]['north'])+','+str(ls_area_d[d]['east'])

'38.014822,114.705913,114.705913,114.719226'

In [60]:
dict_re_b

{'status': 0,
 'message': 'ok',
 'total': 3,
 'result_type': 'city_type',
 'results': [{'name': '张家口市', 'num': 1216},
  {'name': '保定市', 'num': 1206},
  {'name': '石家庄市', 'num': 529}]}

In [33]:
four_loc="38.042437,114.665974,38.097668,114.719226"
url = "https://api.map.baidu.com/place/v2/search"
ak = "hFpVwfn5Q1ZCwMkZ4yd9FyJa96OlH6Li"
scope_status="2"

params = {
    "query":"公司企业",
    "bounds":four_loc,
    "output":"json",
    "ak":ak,
    "tag":"公司企业",
    "extensions_adcode":True,
    "scope":scope_status,
    "ret_coordtype":"gcj02ll",
    "page_size":20,
    "page_num":page_num
}

response1 = requests.get(url=url, params=params)
if response1:
    print(response1.json())

{'status': 0, 'message': 'ok', 'total': 86, 'result_type': 'poi_type', 'results': [{'name': '河北金环模具有限公司', 'location': {'lat': 38.03717800785, 'lng': 114.66004802632}, 'address': '河北省石家庄市裕华区长江大道338号', 'province': '河北省', 'city': '石家庄市', 'area': '裕华区', 'street_id': 'f9fd6062f189e3620f0e6b6b', 'telephone': '(0311)85903751', 'detail': 1, 'uid': 'f9fd6062f189e3620f0e6b6b', 'detail_info': {'tag': '公司企业;公司', 'navi_location': {'lng': 114.66001554323, 'lat': 38.038078051219}, 'type': 'enterprise', 'detail_url': 'http://api.map.baidu.com/place/detail?uid=f9fd6062f189e3620f0e6b6b&output=html&source=placeapi_v2', 'overall_rating': '4.7', 'comment_num': '5', 'shop_hours': '', 'label': '模具公司', 'children': []}}, {'name': '石家庄博世达钢材贸易有限公司', 'location': {'lat': 38.047967548267, 'lng': 114.67841577319}, 'address': '307复线与东三环交叉口向北200米路东庐泰物流园A区2号', 'province': '河北省', 'city': '石家庄市', 'area': '长安区', 'telephone': '(0311)85370383', 'detail': 1, 'uid': 'e50e870bb59246dd82288ac0', 'detail_info': {'tag': '公司企业;公司'

In [52]:
[response.json(),response1.json()]

[{'status': 0,
  'message': 'ok',
  'total': 86,
  'result_type': 'poi_type',
  'results': [{'name': '河北金环模具有限公司',
    'location': {'lat': 38.03717800785, 'lng': 114.66004802632},
    'address': '河北省石家庄市裕华区长江大道338号',
    'province': '河北省',
    'city': '石家庄市',
    'area': '裕华区',
    'street_id': 'f9fd6062f189e3620f0e6b6b',
    'telephone': '(0311)85903751',
    'detail': 1,
    'uid': 'f9fd6062f189e3620f0e6b6b'},
   {'name': '石家庄博世达钢材贸易有限公司',
    'location': {'lat': 38.047967548267, 'lng': 114.67841577319},
    'address': '307复线与东三环交叉口向北200米路东庐泰物流园A区2号',
    'province': '河北省',
    'city': '石家庄市',
    'area': '长安区',
    'telephone': '(0311)85370383',
    'detail': 1,
    'uid': 'e50e870bb59246dd82288ac0'},
   {'name': '河北大有房地产开发有限公司',
    'location': {'lat': 38.037767615087, 'lng': 114.68214496404},
    'address': '河北省石家庄市藁城区安东街8号',
    'province': '河北省',
    'city': '石家庄市',
    'area': '藁城区',
    'street_id': '',
    'telephone': '(0311)88084679',
    'detail': 1,
    'uid': '2123294b51

In [43]:
response1.json()['results'][0]

{'name': '河北金环模具有限公司',
 'location': {'lat': 38.03717800785, 'lng': 114.66004802632},
 'address': '河北省石家庄市裕华区长江大道338号',
 'province': '河北省',
 'city': '石家庄市',
 'area': '裕华区',
 'street_id': 'f9fd6062f189e3620f0e6b6b',
 'telephone': '(0311)85903751',
 'detail': 1,
 'uid': 'f9fd6062f189e3620f0e6b6b',
 'detail_info': {'tag': '公司企业;公司',
  'navi_location': {'lng': 114.66001554323, 'lat': 38.038078051219},
  'type': 'enterprise',
  'detail_url': 'http://api.map.baidu.com/place/detail?uid=f9fd6062f189e3620f0e6b6b&output=html&source=placeapi_v2',
  'overall_rating': '4.7',
  'comment_num': '5',
  'shop_hours': '',
  'label': '模具公司',
  'children': []}}

In [45]:
pd.DataFrame.from_dict(response1.json()['results'])

,name,location,address,province,city,area,street_id,telephone,detail,uid,detail_info
0,河北金环模具有限公司,"{'lat': 38.03717800785, 'lng': 114.66004802632}",河北省石家庄市裕华区长江大道338号,河北省,石家庄市,裕华区,f9fd6062f189e3620f0e6b6b,(0311)85903751,1,f9fd6062f189e3620f0e6b6b,"{'tag': '公司企业;公司', 'navi_location': {'lng': 11..."
1,石家庄博世达钢材贸易有限公司,"{'lat': 38.047967548267, 'lng': 114.67841577319}",307复线与东三环交叉口向北200米路东庐泰物流园A区2号,河北省,石家庄市,长安区,NaN,(0311)85370383,1,e50e870bb59246dd82288ac0,"{'tag': '公司企业;公司', 'navi_location': {'lng': 11..."
2,河北大有房地产开发有限公司,"{'lat': 38.037767615087, 'lng': 114.68214496404}",河北省石家庄市藁城区安东街8号,河北省,石家庄市,藁城区,,(0311)88084679,1,2123294b51351a3946e2745e,"{'tag': '公司企业;公司', 'navi_location': {'lng': 11..."
3,河北笑微微食品有限公司,"{'lat': 38.037732395642, 'lng': 114.68591623333}",河北省石家庄市藁城区开发大街1号,河北省,石家庄市,藁城区,c7eca12b7414c937e115f94c,(0311)83090555,1,c7eca12b7414c937e115f94c,"{'tag': '公司企业;公司', 'navi_location': {'lng': 11..."
4,河北兴达医药有限公司,"{'lat': 38.03777090065, 'lng': 114.68662632177}",河北省石家庄市藁城区开发大街1号,河北省,石家庄市,藁城区,,(0311)89699589,1,0a15c7ec7e7f73fced74f84c,"{'tag': '公司企业;公司', 'navi_location': {'lng': 11..."
5,华瀚通保温管道有限公司,"{'lat': 38.052657207638, 'lng': 114.67861474827}",河北省石家庄市长安区南村镇东三环北,河北省,石家庄市,长安区,NaN,NaN,1,50014af1d20bed8d8c55ec1e,"{'tag': '公司企业;公司', 'navi_location': {'lng': 11..."
6,华能电气公司,"{'lat': 38.040664121705, 'lng': 114.6947159948}",石家庄市藁城区富强大街,河北省,石家庄市,藁城区,,NaN,1,16753f7111a3d768306212e6,"{'tag': '公司企业;公司', 'navi_location': {'lng': 11..."
7,河北安东轻钢彩板有限公司,"{'lat': 38.043473963083, 'lng': 114.66954899699}",河北省石家庄市长安区石津西路中铁十三局集团一工程有限公司西侧,河北省,石家庄市,长安区,0552a759ec1b445326d2bbff,NaN,1,0552a759ec1b445326d2bbff,"{'tag': '公司企业;公司', 'navi_location': {'lng': 11..."
8,爱达木业,"{'lat': 38.044290572057, 'lng': 114.70872099551}",河北省石家庄市藁城区文明路66号,河北省,石家庄市,藁城区,a198c0225f8cb955e5073823,13288712888,1,a198c0225f8cb955e5073823,"{'tag': '公司企业;公司', 'navi_location': {'lng': 11..."
9,霸威龙服装,"{'lat': 38.043645611813, 'lng': 114.70854101373}",文明路北50米,河北省,石家庄市,藁城区,390e0ae0337e6bb54151c099,NaN,1,390e0ae0337e6bb54151c099,"{'tag': '公司企业;公司', 'navi_location': {'lng': 11..."
